In [13]:
#!pip install HydroErr
import HydroErr as HEEM
import numpy 
import pandas as pd
from pandas import read_csv
import math
import keras
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Activation
from sklearn.metrics import mean_squared_error
from keras.layers import BatchNormalization, Dropout
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
AI_Method = "/content/gdrive/My Drive/Colab Notebooks/Nal7/12188MT-1-CNN-STL-Gini-1c"
Var_LRs=[1e-5]# 1e-4,1e-5,1e-6,1e-7,1e-8,1e-9] # -1 kullanma
Var_Decays=[1e-5] #1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]
Var_epochs=[100]# 100,500,1000,1500,2000]#,300,500]
 
Streamflow=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/12188MT-1STL-1.csv', delimiter=';')

In [15]:
import numpy as np
x= Streamflow.drop('Q',axis=1)
Y= Streamflow['Q']
x= x.drop('Wind',axis=1)
 
x= x.drop('Prep',axis=1)
x= x.drop('Temp',axis=1)
 
x.head()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Q-1     237 non-null    float64
 1   Q-2     237 non-null    float64
 2   Q-3     237 non-null    float64
 3   MTI     237 non-null    int64  
 4   Humid   237 non-null    float64
 5   MeanQ   237 non-null    float64
 6   S       237 non-null    float64
 7   T       237 non-null    float64
 8   R       237 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 16.8 KB


In [16]:
X=np.array(x)
y=np.array(Y)
print (X.shape[1])

9


In [17]:
test_size = int(len(X) * 0.15)
valid_size = int(len(X) * 0.15)
train_size= len(X) - (valid_size+test_size)
y_train, y_valid, y_test = y[0:train_size], y[train_size:train_size+valid_size], y[-test_size:]
X_train, X_valid, X_test = X[0:train_size], X[train_size:train_size+valid_size], X[train_size+valid_size:]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [18]:
print (y_train.shape, y_valid.shape, y_test.shape)

(167,) (35,) (35,)


In [19]:
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as geek

In [20]:
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(X.shape[1],1)))
model.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())         
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [21]:
startTime = datetime.now()
fileOfSummary = open(AI_Method+'_Summary_'+str(datetime.now()).replace(":", ".")+'.csv', "w")
fileOfSummary.write("Method,LR,DE,Epoch,RunTime,CCTr,CCVd,CCTt,NSTr,NSVd,NSTt,WITr,WIVd,WITt,RMSETr,RMSEVd,RMSETt,MAETr,MAEVd,MAETt"+str(X.shape[1])+"\n")

112

In [22]:
for e_LR in Var_LRs:
    for e_decay in Var_Decays:
        for e_epoch in Var_epochs:
            
            
            optimizer = keras.optimizers.Adam(lr=e_LR, decay=e_decay)
            model.compile(optimizer=optimizer,loss='mean_squared_error')
        
            print(" ")
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            history = model.fit(X_train, y_train, epochs=e_epoch, batch_size=256, verbose=0, validation_data=(X_valid, y_valid), shuffle=True)
            
            hist_df = pd.DataFrame(history.history) 
            Time_elasped= datetime.now() - startTime
            print('\nTime elapsed: ', Time_elasped)
 
            Train = model.predict(X_train)
            Valid = model.predict(X_valid)
            Test = model.predict(X_test)
 
            FileName=AI_Method+'-LR'+str(e_LR)+'-DE'+str(e_decay)+'-'+str(e_epoch)
            np.savetxt(FileName+'_Train.csv', Train)
            np.savetxt(FileName+'_Valid.csv', Valid)
            np.savetxt(FileName+'_Test.csv', Test)
            #print('a',y_train.shape,Train.shape,len(y_train),len(Train))
            Train = geek.asarray(Train)# np.loadtxt(FileName+'_Train.csv')
            Valid = geek.asarray(Valid)
            Test = geek.asarray(Test)#np.loadtxt(FileName+'_Test.csv')
 
            Train = Train.reshape(Train.shape[0])
            Valid = Valid.reshape(Valid.shape[0])
            Test = Test.reshape(Test.shape[0])
 
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            #print('b',y_train.shape,Train.shape,len(y_train),len(Train))
            print("Train ==>")
            CC_Train=np.corrcoef(y_train,Train)
            print("CC_Train = %.3f" %CC_Train[0,1])
 
            NSTr=HEEM.nse(Train, y_train) #1 - sum((Train-y_train)**2)/sum((y_train-np.mean(y_train))**2)
            print("NSTr = %.2f" %NSTr)
 
            #d = 1 - [ ( sum( (obs - sim)^2 ) ] / sum( ( abs(sim - mean(obs)) + abs(obs - mean(obs)) )^2 ) 

            WITr=HEEM.d(Train, y_train)      #1 - sum((Train-y_train)**2)/sum( ( abs(Train-np.mean(y_train)) + abs(y_train-np.mean(y_train)) )**2 ) 
            print("WITr = %.2f" %WITr)

            rootMeanSquaredErrorTr = sqrt(mean_squared_error(y_train, Train))
            print("RMSE = %.2f" % rootMeanSquaredErrorTr)
            MAETr=mean_absolute_error(y_train, Train)
            print("MAE = %.2f" %  MAETr)
 
            print("Validation ===============>")
            CC_Valid=np.corrcoef(y_valid,Valid)
            print("CC_Test = %.3f" %CC_Valid[0,1])
 
            NSVd=HEEM.nse(Valid, y_valid)
            print("NSTt = %.2f" %NSVd)
 
            WIVd=HEEM.d(Valid, y_valid)
            print("WITr = %.2f" %WITr)

            rootMeanSquaredErrorVd = sqrt(mean_squared_error(y_valid, Valid))
            print("RMSE = %.2f" % rootMeanSquaredErrorVd)
            MAEVd=mean_absolute_error(y_valid, Valid)
            print("MAE = %.2f" %  MAEVd)

            print("Test ======================>")
            CC_Test=np.corrcoef(y_test,Test)
            print("CC_Test = %.3f" %CC_Test[0,1])
 
            NSTt=HEEM.nse(Test, y_test)
            print("NSTt = %.2f" %NSTt)
 
            WITt=HEEM.d(Test, y_test) 
            print("WITr = %.2f" %WITr)

            rootMeanSquaredErrorTt = sqrt(mean_squared_error(y_test, Test))
            print("RMSE = %.2f" % rootMeanSquaredErrorTt)
            MAETt=mean_absolute_error(y_test, Test)
            print("MAE = %.2f" %  MAETt)
 
            fileOfSummary.write(AI_Method+','+str(e_LR)+','+str(e_decay)+','+str(e_epoch)+','+str(Time_elasped)+','
              +str(CC_Train[0,1])+','+str(CC_Valid[0,1])+','+str(CC_Test[0,1])+','
              +str(NSTr)+','+str(NSVd)+','+str(NSTt)+','
              +str(WITr)+','+str(WIVd)+','+str(WITt)+','
              +str(rootMeanSquaredErrorTr)+','+str(rootMeanSquaredErrorVd)+','+str(rootMeanSquaredErrorTt)+','
              +str(MAETr)+','+str(MAEVd)+','+str(MAETt)+'\n')

 
 
LRs: 1e-05
Decays: 1e-05
epochs: 100

Time elapsed:  0:00:04.722853
 
LRs: 1e-05
Decays: 1e-05
epochs: 100
Train ==>
CC_Train = -0.382
NSTr = -0.44
WITr = 0.35
RMSE = 2.83
MAE = 1.51
Validation ===============>
CC_Test = -0.583
NSTt = -0.91
WITr = 0.35
RMSE = 2.71
MAE = 1.80
Test ======================>
CC_Test = -0.431
NSTt = -0.66
WITr = 0.35
RMSE = 1.86
MAE = 1.16


In [23]:
import io
buf = io.StringIO()
x.info(buf=buf)
s = buf.getvalue()

fileOfSummary.write(s)
fileOfSummary.close()

print("Finished")

Finished
